# WGS GVCF samples joint calling, filtering and annotation

Implementing a GATK + ANNOVAR workflow in [SoS](https://github.com/vatlab/SOS), written by Isabelle Schrauwen with software containers built by Gao Wang. 

In [1]:
%revisions -s -n 10

Revision,Author,Date,Message
4762450,haoyueshuai,2020-08-25,update submit_csg
b7958f9,Gao Wang,2020-08-25,Fix a bash variable bug
2986c3c,Gao Wang,2020-08-25,Update documentation
c1da803,Gao Wang,2020-08-25,Add job submission template for CSG cluster
69e450a,Gao Wang,2020-08-24,Add documentation
3b5a1e8,Gao Wang,2020-08-24,Fix ANNOVAR step
43b3150,Gao Wang,2020-08-23,Update joint variant calling pipeline with minimal working example
2cacdc2,Gao Wang,2020-08-20,Remove the need to mount workdir due to recent changes in SoS
afe343c,Gao Wang,2020-08-20,Add variant calling pipeline


## Overview

This SoS workflow notebook contains four workflows:

- `gatk_call`
- `gatk_filter_strict`
- `gatk_filter_basic`
- `vcf_qc`
- `submit_csg`

The first four workflows are for the analysis and the last one is for submitting jobs on the cluster.

All workflow steps are numerically ordered to reflect the execution logic. This is the most straightforward SoS workflow style, the "process-oriented" style. 

## Input data

Samples in `GVCF` format, already indexed:

```
*.gvcf.gz
*.gvcf.gz.tbi
```

To input the list of samples to the workflow, please include all sample file names you would like to analyze, in a text file. For example:

```
GH.AR.SAD.P1.001.0_X3547_S42_1180478_GVCF.hard-filtered.gvcf.gz
GH.AR.SAD.P1.003.0_92455_S43_1189700_GVCF.hard-filtered.gvcf.gz
GH.AR.SAD.P1.004.0_92456_S44_1189701_GVCF.hard-filtered.gvcf.gz
GH.AR.SAD.P1.005.0_92457_S20_1189702_GVCF.hard-filtered.gvcf.gz
...
```

and save it as, eg, `20200820_sample_manifest.txt`. This text file will be the input file to the pipeline.

## Reference data preparation

Human genome reference files are needed for `GATK` joint calling; `ANNOVAR` database references are needed for `ANNOVAR` annotations.

- `GATK` reference files include:

```
*.fa
*.fa.fai
*.dict
```

- `ANNOVAR` reference files ship with `ANNOVAR` software, under a folder called `humandb`.

This workflow assumes that the required files already exit. This pipeline does not provide steps to download or to generate them automatically, which you could find in the tutorial slides. The pipeline will indeed check the availability of the reference files and quit on error if they are missing.

## Run the workflow

The workflow is currently designed to run on a Linux cluster (via `singularity`) although it can also be executed on a Mac computer
(via `docker`). In brief, after installing [SoS](https://github.com/vatlab/SOS) (also see section "Software Configuration" below), 
you can choose to run different workflows modules.

For example to run the variant calling workflow,

```
sos run gatk_joint_calling.ipynb call \
    --vcf-prefix /path/to/some_vcf_file_prefix \
    --samples /path/to/list/of/sample_gvcf.txt \
    --samples-dir /path/to/sample_gvcf \
    --ref-genome /path/to/reference_genome.fa \
    ...
```

to run variant filtering, 


```
sos run gatk_joint_calling.ipynb filter \
    --vcf-prefix /path/to/some_vcf_file_prefix \
    ...
```

to run annotation,

```
sos run gatk_joint_calling.ipynb annovar \
    --vcf-prefix /path/to/some_vcf_file_prefix \
    ...
```

You can put all these 3 commands to one bash file and execute that, so you run all steps one after another.

Note that `...` are additional options that fall into two categories:

1. Options needed to run the bioinformatics steps (e.g. ref_genome)
2. Options needed for SoS to run on different platforms ( e.g. container-option)

To view all options,

In [8]:
sos run gatk_joint_calling.ipynb -h

usage: sos run gatk_joint_calling.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  call
  strict_filter
  basic_filter
  vcf_qc

Global Workflow Options:
  --vcf-prefix joint_call_output (as path)
                        Combined VCF file prefix, including path to the output
                        but without vcf.gz extension, eg
                        "/path/to/output_filename".
  --cwd VAL (as path, required)
                        Working directory
  --build hg19
                        Human genome build
  --vcf-filter strict
                        VCF filtering strategy e.x: strict or basic (default is
                        strict)
  --mem 12 (as int)
  

Please read these options carefully before you start running the analysis.

## Minimal working example

A minimal example data-set can be found on CSG cluster. The following commands use this data-set, although in practice you should change the paths to point to your own data of interest.

Joint calling:

```
sos run gatk_joint_calling.ipynb call \
    --container-option /mnt/mfs/statgen/containers/gatk4-annovar.sif \
    --vcf-prefix output/minimal_example \
    --samples /mnt/mfs/statgen/data_private/gatk_joint_call_example/20200820_sample_manifest.txt \
    --samples-dir /mnt/mfs/statgen/data_private/gatk_joint_call_example/ \
    --ref-genome /mnt/mfs/statgen/isabelle/REF/refs/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly.fa \
    --cwd output \
    --vcf_filter strict
```

Filtering with strict filters:

```
sos run gatk_joint_calling.ipynb strict_filter \
    --container-option /mnt/mfs/statgen/containers/gatk4-annovar.sif \
    --vcf-prefix output/minimal_example \
    --cwd output \
    --vcf_filter strict
```

Filtering with basic filters:

```
sos run gatk_joint_calling.ipynb basic_filter \
    --container-option /mnt/mfs/statgen/containers/gatk4-annovar.sif \
    --vcf-prefix output/minimal_example \
    --ref-genome /mnt/mfs/statgen/isabelle/REF/refs/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly.fa\
    --cwd output \
    --vcf_filter strict
```

VCF quality control (sex checks, IBD, heterozygosity, etc):

```
sos run gatk_joint_calling.ipynb vcf_qc \
    --container-option /mnt/mfs/statgen/containers/gatk4-annovar.sif \
    --vcf-prefix output/minimal_example.snp_indel.filter.PASS \
    --cwd output \
    --vcf_filter strict
    
```

## Share the workflow with people

Use 

```
sos convert gatk_joint_calling.ipynb gatk_joint_calling.html --template sos-cm-toc
```

to convert this workflow to an HTML file, then pass it around to others to read it.

## Software configuration

Instructions on SoS and docker installation can be found on [our CSG wiki](http://statgen.us/lab-wiki/orientation/jupyter-setup.html). 
The instructions works for both Mac and Linux, unless otherwise specified.

## Global parameter settings

In [3]:
[global]
# Combined VCF file prefix, including path to the output but without vcf.gz extension, 
# eg "/path/to/output_filename".
parameter: vcf_prefix = path('joint_call_output')
# Working directory
parameter: cwd = path
# Human genome build
parameter: build = 'hg19'
# VCF filtering strategy e.x: strict or basic (default is strict)
parameter: vcf_filter = 'strict'
# Memory allocated to a job, in terms of Gigabyte
parameter: mem=12
# Software container option
parameter: container_option = 'gaow/gatk4-annovar'

## Joint variant calling from GVCF files

In [9]:
# Combine GVCF files
[call_1]
# A file listing out all sample GVCF you would like to analyze. 
# Each line is one sample GVCF name.
parameter: samples = path
# Directory where sample GVCF files locate.
parameter: samples_dir = path()
# Path to reference genome file
parameter: ref_genome = path('refs/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly.fa')
#
fail_if(not samples.is_file(), msg = 'Need valid sample name list file input via ``--samples`` option!')
import os
sample_files = [f'{samples_dir}/{os.path.basename(x.strip())}' for x in open(samples).readlines()]
for x in sample_files:
    fail_if(not path(x).is_file(), msg = f'Cannot find file ``{x}``. Please use ``--samples-dir`` option to specify the directory for sample files.')
    fail_if(not x.endswith('gvcf.gz'), msg = f'Input file ``{x}`` does not have ``.gvcf.gz`` extension.')
fail_if(len(sample_files) == 0, msg = 'Need at least one input sample file!')
fail_if(not ref_genome.is_file(), msg = f'Cannot find reference genome ``{ref_genome}``. Please use ``--ref-genome`` option to specify it.')
fail_if(not path(f"{ref_genome:a}.fai").is_file(), msg = f'Cannot find reference genome index file ``{ref_genome}.fai``. Please make sure it exists.')
fail_if(not path(f"{ref_genome:an}.dict").is_file(), msg = f'Cannot find reference genome dict file ``{ref_genome:n}.dict``. Please make sure it exists.')

depends: system_resource(mem = f'{mem}G'), ref_genome
input: sample_files
output: f'{vcf_prefix:a}.combined.vcf.gz'

bash: container=container_option, volumes=[f'{ref_genome:ad}:{ref_genome:ad}'], expand="${ }", stderr=f'{_output:n}.err', stdout=f'{_output:n}.out'
    ${'&&'.join(["tabix -p vcf %s" % x for x in _input if not path(x + '.tbi').is_file()])}
    gatk --java-options "-Xmx${mem}g" CombineGVCFs \
        -R ${ref_genome} \
        ${' '.join(['--variant %s' % x for x in _input])} \
        -O ${_output}

In [6]:
# Joint calling
[call_2]
# Path to reference genome file
parameter: ref_genome = path
output: f'{vcf_prefix:a}.vcf.gz'


bash: container=container_option, volumes=[f'{ref_genome:ad}:{ref_genome:ad}'], expand="${ }", stderr=f'{_output:nn}.err', stdout=f'{_output:nn}.out'
    gatk --java-options "-Xmx${mem}g" GenotypeGVCFs \
        -R ${ref_genome} \
        -V ${_input} \
        -O ${_output}

## Variant filtering

Since we have two types of variants SNP and Indels, the first two steps of the filter workflow pipeline process the two variant types in parallel, then merge them and do additional filtering wiht steps 3 and 4.

### Strict filter

In [10]:
# Split into SNP and INDEL for separate PASS filters
[strict_filter_1]
variant_type = ['SNP', 'INDEL']
input: f'{vcf_prefix:a}.vcf.gz', for_each='variant_type', concurrent = True
output: f'{vcf_prefix:a}.{_variant_type.lower()}.vcf.gz'


bash: container=container_option, expand="${ }", stderr=f'{_output:nn}.err', stdout=f'{_output:nn}.out'
    gatk --java-options '-Xmx${mem}g' SelectVariants \
        -V ${_input} \
        -select-type ${_variant_type} \
        -O ${_output}

In [10]:
# PASS or filter for indels and SNPs (Note | not recommended for filters)
# Ignore MQRankSum warnings <- can only be calculated for het sites (not homs)
[strict_filter_2]
parameter: snp_filters = ['QD < 2.0, QD2', 'QUAL < 30.0, QUAL30', 'SOR > 3.0, SOR3', 'FS > 60.0, FS60', 'MQ < 40.0, MQ40', 'MQRankSum < -12.5, MQRankSum-12.5', 'ReadPosRankSum < -8.0, ReadPosRankSum-8']
parameter: indel_filters = ["QD < 2.0, QD2", "QUAL < 30.0, QUAL30", "FS > 200.0, FS200", "ReadPosRankSum < -20.0, ReadPosRankSum-20"]
input: paired_with = dict(filter_option=[snp_filters, indel_filters])
output: f'{_input:nn}.filter.vcf.gz'


bash: container=container_option, expand="${ }", stderr=f'{_output:nn}.err', stdout=f'{_output:nn}.out'
    gatk --java-options '-Xmx${mem}g' VariantFiltration \
        -V ${_input} \
        ${" ".join(['-filter "%s" --filter-name "%s"' % tuple([y.strip() for y in x.split(',')]) for x in _input.filter_option])} \
        -O ${_output}

In [ ]:
# Merge back SNP and INDEL
[strict_filter_3]
input: group_by = 'all'
output: f'{vcf_prefix:a}.snp_indel.filter.vcf.gz'


bash: container=container_option, expand="${ }", stderr=f'{_output:nn}.err', stdout=f'{_output:nn}.out'
    gatk --java-options '-Xmx${mem}g' MergeVcfs \
     -I ${_input[0]} -I ${_input[1]} -O ${_output}

In [ ]:
# remove non-PASS variants if wanted
[strict_filter_4]
output: strict_out= f'{vcf_prefix:a}.snp_indel.filter.strict_QC.PASS.vcf.gz'


bash: container=container_option, expand="${ }", stderr=f'{_output:nn}.err', stdout=f'{_output:nn}.out'
    gatk --java-options '-Xmx${mem}g' SelectVariants \
        -V ${_input} -O ${_output} \
        --exclude-filtered

### Basic filter

In [ ]:
# remove all coverage < 4x, strand bias and end of read bias
[basic_filter_1]
# Path to reference genome file
parameter: ref_genome = path('refs/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly.fa')
parameter: variant_filter = ['QUAL < 30.0 , QUAL30', 'FS > 200.0, FS200', 'ReadPosRankSum < -20.0, ReadPosRankSum-20', 'DP < 4, DP4']
input: f'{vcf_prefix:a}.vcf.gz'
output: f'{_input:nn}.snp_indel.filter.basic_QC.vcf.gz'
bash: container=container_option, expand="${ }", stderr=f'{_output:nn}.err', stdout=f'{_output:nn}.out'
    gatk --java-options '-Xmx${mem}g' VariantFiltration \
    -R ${ref_genome} \
    -V ${_input} \
    ${" ".join(['-filter "%s" --filter-name "%s"' % tuple([y.strip() for y in x.split(',')]) for x in variant_filter])} \
    -O ${_output}
    

In [ ]:
# Remove non-PASS variants
[basic_filter_2]
output: basic_out=f'{_input:nn}.PASS.vcf.gz'

bash: container=container_option, expand="${ }", stderr=f'{_output:nn}.err', stdout=f'{_output:nn}.out'
    gatk --java-options '-Xmx${mem}g' SelectVariants \
        -V ${_input} -O ${_output} \
        --exclude-filtered

## Extra VCF QC filters

In [ ]:
# QC VCF for relatedness
[vcf_qc_1 (check relatedness)]
input: f"{vcf_prefix:a}.snp_indel.filter.{vcf_filter}_QC.PASS.vcf.gz" 
output: f'{cwd}/vcf_qc/{_input:bnn}.relatedness', f'{cwd}/vcf_qc/{_input:bnn}.relatedness2'
bash: expand="${ }", stderr=f'{cwd}/vcf_qc/{_output[0]:b}.err', stdout=f'{cwd}/vcf_qc/{_output[0]:b}.log'

    vcftools --relatedness --gzvcf ${_input} --out ${_output[0]:n}
    vcftools --relatedness2 --gzvcf ${_input} --out ${_output[1]:n}

In [ ]:
# QC VCF for sex check
[vcf_qc_2 (check sex)]
input: f"{vcf_prefix:a}.snp_indel.filter.{vcf_filter}_QC.PASS.vcf.gz"
output: f"{cwd}/vcf_qc/{_input:bnn}.bed", f"{cwd}/vcf_qc/{_input:bnn}.sex.sexcheck", f"{cwd}/vcf_qc/{_input:bnn}.sex2.sexcheck"
bash: expand="${ }", stderr=f"{_output[1]:n}.err",stdout=f"{_output[1]:n}.log"
    plink --vcf ${_input} --double-id --make-bed --out ${_output[0]:n} --allow-extra-chr
    plink --bfile ${_output[0]:n} --check-sex --out ${_output[1]:n} --allow-extra-chr
    plink --bfile ${_output[0]:n} --check-sex 0.35 0.65 --out ${_output[2]:n} --allow-extra-chr
    rm ${_output[1]:n}.nosex && rm ${_output[2]:n}.nosex && rm ${_output[1]:n}.log && rm ${_output[2]:n}.log && rm ${_output[0]:n}.nosex

In [ ]:
# QC VCF for IBD
[vcf_qc_3 (IBD)]
input: f"{cwd}/vcf_qc/{vcf_prefix:b}.snp_indel.filter.{vcf_filter}_QC.PASS.bed" 
output: f'{_input:n}.IBD.genome',
        f'{_input:n}.HET.het',
        f'{_input:n}.IBC.ibc',
        f'{_input:n}.SEX.2.C.sexcheck',
        vcf=f'{_input:n}.C.VCF.vcf'
bash: expand="${ }", stderr=f'{_output[0]}.err', stdout=f'{_output[0]}.log'
    #add plink IBD
    #missing rate per SNP MAF and HWE cut-off
    plink --bfile ${_input:n} --geno 0.1 --hwe 0.00001 --maf 0.05 --make-bed --out ${_input:n}.C --allow-extra-chr
    #LD pruning with window size 100 step size 10 and r^2 threshold 0.5 (MAF <0.05)
    plink --bfile ${_input:n}.C --indep-pairwise 50 5 0.5 --make-bed --out  ${_input:n}.CP --allow-extra-chr
    #IBD sharing
    plink --bfile ${_input:n}.CP --genome --make-bed --out ${_output[0]:n} --allow-extra-chr
    #het (Inbreeding and absence of heterozygosity)
    plink --bfile ${_input:n}.CP --het --make-bed --out ${_output[1]:n} --allow-extra-chr
    #IBCs (Inbreeding coeff)
    plink --bfile ${_input:n}.CP --ibc --make-bed --out ${_output[2]:n} --allow-extra-chr
    ###cleaned sex
    plink --bfile ${_input:n}.C --check-sex 0.35 0.65 --out ${_output[3]:n} --allow-extra-chr
    ####cleaned relatedness
    plink --bfile ${_input:n}.C --recode vcf --out ${_output[4]:n} --allow-extra-chr


In [ ]:
# QC VCF for relatedness
[vcf_qc_4 (vcftools)]
input: named_output("vcf")
output: f'{_input:n}.C.relatedness', f'{_input:n}.C.2.relatedness2'
bash: expand="${ }", stderr=f'{_output[0]}.err', stdout=f'{_output[0]}.log'

    bgzip ${_input} && tabix -p vcf ${_input}.gz
    vcftools --relatedness --gzvcf ${_input}.gz --out ${_output[0]:n}
    vcftools --relatedness2 --gzvcf ${_input}.gz --out ${_output[1]:n}

In [ ]:
# QC VCF for homozygosity mapping 
[vcf_qc_5 (homozygosity mapping)]
parameter: vcf_filter = 'strict'
input:  f"{cwd}/vcf_qc/{vcf_prefix:b}.snp_indel.filter.{vcf_filter}_QC.PASS.bed"
output: f'{_input:n}.HOM.hom'
bash: expand="${ }", stderr=f'{_output:nn}.err', stdout=f'{_output:nn}.log'
    ##hom_mapping per sample (at least 100 SNPs, and of total length ≥ 1000 (1Mb) - 0.01 MAF
    plink --bfile ${_input:n} --geno 0.1 --hwe 0.00001 --maf 0.01 --make-bed --out ${_input:n}.CH --allow-extra-chr
    plink --bfile ${_input:n}.CH --homozyg --make-bed --out ${_output:n} --allow-extra-chr
    #remove all the unwanted files at the end
    ##FIXME
    mkdir ${cwd}/vcf_qc/cache
    mv ${cwd}/vcf_qc/*.{bed,bim,fam,log,nosex,in,out,gz,tbi} ${cwd}/vcf_qc/cache


## Submit jobs to the cluster

Suppose we would like to submit these lines of commands to the cluster:

```
sos run gatk_joint_calling.ipynb call \
    --container-option /mnt/mfs/statgen/containers/gatk4-annovar.sif \
    --vcf-prefix output/minimal_example \
    --samples /mnt/mfs/statgen/data_private/gatk_joint_call_example/20200820_sample_manifest.txt \
    --samples-dir /mnt/mfs/statgen/data_private/gatk_joint_call_example/ \
    --ref-genome /mnt/mfs/statgen/isabelle/REF/refs/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly.fa\
    --cwd output\ \
    --variant_filter 'strict'

sos run gatk_joint_calling.ipynb strict_filter \
    --vcf-prefix output/minimal_example \
    --cwd output/\
    --variant_filter 'strict'
    
sos run gatk_joint_calling.ipynb basic_filter \
    --vcf-prefix output/minimal_example \
    --cwd output/\
    --variant_filter 'basic'

module load Singularity
module load VCFTOOLS/0.1.17
module load PLINK/1.9.10
sos run gatk_joint_calling.ipynb vcf_qc \
    --vcf-prefix output/minimal_example \
    --cwd output/\
    --variant_filter 'basic'
    
    
```

First, we save the above lines to a text file, e.g. call it `analysis_commands_20200825.txt`, then use the following workflow steps to allocate resources and submit the jobs.

Example to submit a job:

```
sos run gatk_joint_calling.ipynb submit_csg \
    --cmd_file command_1027.txt \
    --cwd output
    
sos run ~/gatk_joint_calling_test.ipynb submit_csg \
    --cmd_file ~/gatk_joint_calling/command_1027.txt \
    --cwd output
```


If you want to run in a dryrun mode, meaning just simply test the process but do not genrate results
```
sos run gatk_joint_calling.ipynb submit_csg \
    --cmd_file analysis_commands_20200825.txt \
    --cwd output \
    --dryrun True
```

In [ ]:
# Job submission on CSG cluster
[submit_csg]
# Path to job file
parameter: cmd_file=path
# Total run time allocated to the script
parameter: time='36:00:00'
parameter: dryrun = False
input: cmd_file
python3: expand = '$[ ]'
    tpl = '''
    #!/bin/sh
    #$ -l h_rt=$[time]
    #$ -l h_vmem=$[mem+6]G
    #$ -N gatk_joint_call
    #$ -cwd
    #$ -j y
    #$ -S /bin/bash
    module load Singularity
    export PATH=$HOME/miniconda3/bin:$PATH
    set -e
    '''
    script = tpl.lstrip() + ''.join(open($[_input:r]).readlines())
    exe = 'cat' if $[dryrun] else 'qsub'
    from subprocess import Popen, PIPE
    import sys
    p = Popen(exe, shell = False, stdin = PIPE, stdout = PIPE, stderr = PIPE, close_fds = True)
    for item in p.communicate(script.encode(sys.getdefaultencoding())):
        output = item.decode(sys.getdefaultencoding()).rstrip()
        if output:
            print(output)